#Imports

In [ ]:
import os
import json
import gdown
import random
import shutil
import numpy as np

import torch
from tqdm import tqdm
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.models import vit_b_16, ViT_B_16_Weights

from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight

from datasets import load_dataset
from huggingface_hub import login
login()

os.environ["HF_TOKEN"] = ""

#Dataset preparation

Download Emoset dataset and prepare it for training. Extract relevant emotions and merge some that are needed. Create folders with images of different emotions and split new dataset into train/valid/test.

In [ ]:
!wget -O EmoSet-118K.zip "https://www.dropbox.com/scl/fi/myue506itjfc06m7svdw6/EmoSet-118K.zip?rlkey=7f3oyjkr6zyndf0gau7t140rv&dl=1"

In [ ]:
!unzip -q -o "/content/EmoSet-118K.zip" -d "/content/EmoSet-118K_extracted"

In [ ]:
#Extract images of different emotions
KEEP_AWE_ANGER = True
MAX_PER_CLASS = 10500

# root folder
ROOT = "/content/EmoSet-118K_extracted"
ANNOT_DIR = os.path.join(ROOT, "annotation")
IMAGE_DIR = os.path.join(ROOT, "image")

# output folder
!rm -rf "/content/EmoSet_balanced"
OUT_DIR = "/content/EmoSet_balanced"
os.makedirs(OUT_DIR, exist_ok=True)

def map_emotion(em):
    mapping = {
        "anger": "angry",
        "disgust": "disgust",
        "fear": "fear",
        "sadness": "sad",
        "amusement": "happy",
        "excitement": "happy",
    }

    if KEEP_AWE_ANGER:
        mapping["awe"] = "surprised"
        mapping["anger"] = "angry"
    else:
        mapping["awe"] = None
        mapping["anger"] = None

    mapping["contentment"] = None  # always excluded not relevant to our dataset

    return mapping.get(em, None)


#load all annottions
all_records = []

for emotion_folder in os.listdir(ANNOT_DIR):
    full_emotion_dir = os.path.join(ANNOT_DIR, emotion_folder)

    if not os.path.isdir(full_emotion_dir):
        continue

    for json_name in os.listdir(full_emotion_dir):
        if not json_name.endswith(".json"):
            continue

        json_path = os.path.join(full_emotion_dir, json_name)

        try:
            with open(json_path, "r") as f:
                entry = json.load(f)
        except:
            continue

        all_records.append(entry)

print("Loaded annotation JSONs:", len(all_records))


#Group images by every target class
grouped = {}

for rec in all_records:
    original_em = rec["emotion"]
    mapped = map_emotion(original_em)

    if mapped is None:
        continue

    img_id = rec["image_id"]
    img_path = os.path.join(IMAGE_DIR, original_em, img_id + ".jpg")

    if not os.path.exists(img_path):
        continue

    grouped.setdefault(mapped, []).append(img_path)


#Balance sampling when merging different emotions
final_dataset = {}

for cls, paths in grouped.items():

    #Merge for happy
    if cls == "happy":
        amusement_paths  = [p for p in paths if "/amusement/"  in p]
        excitement_paths = [p for p in paths if "/excitement/" in p]

        print(f"\nHappy breakdown:")
        print(f"  amusement:  {len(amusement_paths)}")
        print(f"  excitement: {len(excitement_paths)}")

        half = MAX_PER_CLASS // 2

        random.shuffle(amusement_paths)
        random.shuffle(excitement_paths)

        selected_amusement  = amusement_paths[:half]
        selected_excitement = excitement_paths[:half]

        selected = selected_amusement + selected_excitement
        random.shuffle(selected)

        final_dataset[cls] = selected
        print(f"happy: selected {len(selected)} balanced samples\n")
        continue

    #take randomely for other emotions
    random.shuffle(paths)
    selected = paths[:MAX_PER_CLASS]

    final_dataset[cls] = selected
    print(f"{cls}: {len(selected)} images selected")


#Copy file to new dataset
print("\nCopying images into /content/EmoSet_balanced ...\n")

for cls, paths in final_dataset.items():
    out_dir = os.path.join(OUT_DIR, cls)
    os.makedirs(out_dir, exist_ok=True)

    for img_path in tqdm(paths, desc=f"Copying {cls}", total=len(paths)):
        shutil.copy(img_path, os.path.join(out_dir, os.path.basename(img_path)))

print("\nBalanced dataset creation complete.")
print("Final dataset stored at:", OUT_DIR)

# Summary for each class
print("\nSummary:")
for cls, imgs in final_dataset.items():
    print(f"{cls}: {len(imgs)} images")


In [ ]:
#Split dataset ubti train/test/valid
SRC = "/content/EmoSet_balanced"
DEST = "/content/dataset_new"
SPLITS = ["train", "valid", "test"]

# Ratios
train_ratio = 0.8
val_ratio   = 0.1
test_ratio  = 0.1

# Make folders
for split in SPLITS:
    os.makedirs(os.path.join(DEST, split), exist_ok=True)

# Process each emotion folder
for emotion in os.listdir(SRC):
    src_folder = os.path.join(SRC, emotion)
    if not os.path.isdir(src_folder):
        continue

    print(f"\n📁 Processing class: {emotion}")

    # list and shuffle images
    images = os.listdir(src_folder)
    random.shuffle(images)

    n = len(images)
    n_train = int(n * train_ratio)
    n_val   = int(n * val_ratio)

    train_imgs = images[:n_train]
    val_imgs   = images[n_train:n_train+n_val]
    test_imgs  = images[n_train+n_val:]

    # Create class folders
    for split in SPLITS:
        os.makedirs(os.path.join(DEST, split, emotion), exist_ok=True)

    # Copy images
    for img in tqdm(train_imgs, desc=f"{emotion} → train"):
        shutil.copy(os.path.join(src_folder, img),
                    os.path.join(DEST, "train", emotion, img))

    for img in tqdm(val_imgs, desc=f"{emotion} → valid"):
        shutil.copy(os.path.join(src_folder, img),
                    os.path.join(DEST, "valid", emotion, img))

    for img in tqdm(test_imgs, desc=f"{emotion} → test"):
        shutil.copy(os.path.join(src_folder, img),
                    os.path.join(DEST, "test", emotion, img))

print(f"\nSplit complete. Dataset ready at {DEST}")

#Train Model

We will apply transformations for best compatibility with model, setup data loaders, load model, and finally train it. While training we always check validation and test dataset.

In [ ]:
#Transform and setup test loaders
base_dir = "/content/dataset_new"
train_dir = os.path.join(base_dir, "train")
val_dir   = os.path.join(base_dir, "valid")
test_dir  = os.path.join(base_dir, "test")

#Params for training
batch_size = 64
epochs = 50
lr = 5e-5

!mkdir "/content/checkpoints"
ckpt_local_dir = "/content/checkpoints"
os.makedirs(ckpt_local_dir, exist_ok=True)


SAVE_BEST_TEST = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Device: {device}")
print(f"Model: ViT-Base-16 (SWAG Weights) | Input Size: 384x384 | Batch: {batch_size}")


train_transform = transforms.Compose([
    transforms.Resize((384, 384)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_test_transform = transforms.Compose([
    transforms.Resize((384, 384)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


train_ds = datasets.ImageFolder(train_dir, transform=train_transform)
val_ds   = datasets.ImageFolder(val_dir,   transform=val_test_transform)
test_ds  = datasets.ImageFolder(test_dir,  transform=val_test_transform)

class_names = list(train_ds.class_to_idx.keys())
num_classes = len(class_names)
print(f"✅ Classes ({num_classes}):", train_ds.class_to_idx)


train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True,  num_workers=2, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)
test_loader  = DataLoader(test_ds,  batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

print(train_ds.classes, len(train_ds))
print(val_ds.classes,   len(val_ds))
print(test_ds.classes,  len(test_ds))

In [ ]:
#Setup models, load pretrained weights, and change classification head.
train_labels = [lbl for _, lbl in train_ds.samples]
cls_w = compute_class_weight("balanced", classes=np.unique(train_labels), y=train_labels)
criterion = nn.CrossEntropyLoss(weight=torch.FloatTensor(cls_w).to(device))

print("Loading SWAG Weights")
weights = ViT_B_16_Weights.IMAGENET1K_SWAG_E2E_V1
model = vit_b_16(weights=weights)

in_features = model.heads.head.in_features
model.heads.head = nn.Linear(in_features, num_classes)

model = model.to(device)

#Optimizer and Scheduler
optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=0.05)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

In [ ]:
#Training and Evaluation
@torch.no_grad()
def evaluate(loader, name):
    model.eval()
    tot_loss = tot_correct = tot = 0
    all_preds, all_labels = [], []

    for x, y in loader:
        x, y = x.to(device, non_blocking=True), y.to(device, non_blocking=True)
        out = model(x)
        loss = criterion(out, y)

        pred = out.argmax(1)
        tot_loss += loss.item() * x.size(0)
        tot_correct += (pred == y).sum().item()
        tot += y.size(0)

        all_preds.extend(pred.cpu().numpy())
        all_labels.extend(y.cpu().numpy())

    avg_loss = tot_loss / tot if tot else 0.0
    acc = tot_correct / tot if tot else 0.0

    print(f"{name} → Loss: {avg_loss:.4f} | Acc: {acc:.4f}")
    print(classification_report(all_labels, all_preds, target_names=class_names, digits=4))
    return avg_loss, acc

def save_ckpt(path, epoch, tag):
    torch.save({
        "epoch": epoch,
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "scheduler_state_dict": scheduler.state_dict(),
        "tag": tag,
    }, path)
    print(f"Saved Checkpoint: {path}")



best_val_acc = 0.0
best_test_acc = 0.0

print("\nStart Training Loop...")

for epoch in range(1, epochs + 1):
    print(f"\nEpoch {epoch}/{epochs}")

    model.train()
    run_loss = run_correct = run_total = 0

    for x, y in tqdm(train_loader, desc="Training"):
        x, y = x.to(device, non_blocking=True), y.to(device, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()

        run_loss += loss.item() * x.size(0)
        run_correct += (out.argmax(1) == y).sum().item()
        run_total += x.size(0)

    scheduler.step()
    train_acc = run_correct / run_total
    print(f"Train → Loss: {run_loss/run_total:.4f} | Acc: {train_acc:.4f}")


    _, val_acc  = evaluate(val_loader,  "Val")
    _, test_acc = evaluate(test_loader, "Test (Monitoring)")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_local = os.path.join(ckpt_local_dir, "vit_swag_best_val.pt")

        save_ckpt(best_local, epoch, tag=f"best_val_acc_{best_val_acc:.4f}")
        print(f"New Best Val Model ({best_val_acc:.4f}) saved to Drive.")

        with open(os.path.join(ckpt_local_dir, "best_val_metrics.txt"), "w") as f:
            f.write(f"epoch={epoch}, val_acc={best_val_acc:.6f}, test_acc={test_acc:.6f}\n")


    if SAVE_BEST_TEST and test_acc > best_test_acc:
        best_test_acc = test_acc
        best_test_local = os.path.join(ckpt_local_dir, "vit_swag_best_test_LEAKY.pt")

        save_ckpt(best_test_local, epoch, tag=f"best_test_acc_{best_test_acc:.4f}")
        print(f"New Best Test Model ({best_test_acc:.4f}) saved to Drive.")

print("\nAll Process Complete!")
print(f"Final Best Val Acc: {best_val_acc:.4f}")
if SAVE_BEST_TEST:
    print(f"Final Best Test Acc (Leaky): {best_test_acc:.4f}")